# Classification

Nosso principal objetivo na modelagem é realizar uma clusterização dos dados para posteriomente criar um sistema descritivo, ou seja, nossa meta é criar visualizações que possam ser úteis para os médicos.

Assim, mesmo que os nossos dados possuam rótulos devidamente prontos nós tentaremos utilizar a clusterização. Contudo, inicialmente iremos utilizar algoritmos supervisionados para averiguar se os dados na forma como estão conseguem distinguir bem as classes.

## 1. Leitura dos dados

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [2]:
#Setando diretorio Data como o atual
os.chdir('../Data')

#Leitura dos dados
# df_parkinson = pd.read_csv('parkinson_normalizado_ss.csv',index_col='name')
# df_parkinson = pd.read_csv('parkinson_normalizado_mm.csv',index_col='name')
df_parkinson = pd.read_csv('parkinson_pca.csv',index_col='name')

#Mapeando dados para conseguir calcular o AUC
df_parkinson['drug'] = df_parkinson['drug'].map({'CBD':1,'placebo':0})

# 2. Modelagem

## 2.1 Classificação

Iremos realizar a modelagem com 3 algoritmos supervisionados:

- Árvore de Decisão
- Random Forest
- Regressão Logística

Como principal métrica de avaliaçã iremos utilizar o AUC (Área Under Curve) por nosso problema ser binário. De qualquer forma também iremos avaliar acurácia, sensibilidade, especificidade e erros para cada pessoa.

Por fim, para garantir o poder generativo dos nossos algoritmos será utilizado uma variação do k-fold.

Gostaríamos de lembrar que será utilizado os dados obtidos após a aplicação do PCA.

### 2.1.1 Sem variáveis categórias

In [3]:
'''Realiza uma avaliação da classificação'''
def evaluate_classifier(X,Y,split=1,type_clf='dt'):
    
    #Pessoas do dataset
    unique_people = X.index.unique()

    #Listas para armazenar os resultados finais
    list_score = []
    list_sens = []
    list_spec = []
    list_auc = []
    list_error = []

    for i in range(500):
        if(split == 1):
            #Sample das pessoas
            people_train = np.random.choice(unique_people,size=11,replace=False)
            people_test = unique_people[~unique_people.isin(people_train)]
    
            #Criando treino e test
            X_train = X.loc[people_train]
            Y_train = Y.loc[people_train]
            X_test = X.loc[people_test]
            Y_test = Y.loc[people_test]
        elif(split == 2):
            X_train,X_test,Y_train,Y_test = train_test_split(X,Y,stratify=Y,test_size=0.4)

        #Criando classificador
        if(type_clf == 'dt'):
            clf = DecisionTreeClassifier().fit(X_train,Y_train)
        elif(type_clf == 'rf'):
            clf = RandomForestClassifier(n_estimators=100).fit(X_train,Y_train)
        else:
            clf = LogisticRegression(solver='liblinear',penalty='l1')

        #Matrix de confusao
        tn, fp, fn, tp = confusion_matrix(Y_test,clf.predict(X_test)).ravel()

        #Sensibilidade
        sens = tp/(tp+fn)
        
        #Especificidade
        spec = tn/(tn+fp)

        #Score
        score = (tn+tp)/(tn+tp+fn+fp)
        
        #AUC
        auc = roc_auc_score(Y_test,clf.predict(X_test))
        
        #Pessoas que erraram
        error = (clf.predict(X_test) == Y_test)
        list_error = np.append(list_error,error[error == False].index.unique().values)
        list_score.append(score)
        list_sens.append(sens)
        list_spec.append(spec)
        list_auc.append(auc)

    print('AUC:',np.nanmean(list_auc))
    print('Score:',np.nanmean(list_score))
    print('Sensibilidade:',np.nanmean(list_sens))
    print('Especificidade:',np.nanmean(list_spec),'\n')
    
    #Calculo do erro por pessoa
    error = pd.Series(list_error).value_counts()
    print(error/error.values.sum())

In [4]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float')
Y = df_parkinson['drug']

#Classificacao
evaluate_classifier(X,Y,split=2)

AUC: 0.517
Score: 0.517
Sensibilidade: 0.5176129032258063
Especificidade: 0.5163870967741937 

person_21    0.047114
person_11    0.047018
person_4     0.046827
person_8     0.046732
person_0     0.046636
person_12    0.046636
person_20    0.046541
person_18    0.046254
person_10    0.046158
person_16    0.046063
person_17    0.045967
person_19    0.045776
person_7     0.045585
person_5     0.045585
person_2     0.045298
person_3     0.044629
person_9     0.044343
person_13    0.044151
person_1     0.044151
person_14    0.043865
person_15    0.042336
person_6     0.042336
dtype: float64


### 2.1.2 Variáveis categórias sem ser dummy

In [5]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()
Y = df_parkinson['drug']

#Adicionando variaveis categoricas
X['measure'] = LabelEncoder().fit_transform(df_parkinson['measure'])
X['evaluate'] = df_parkinson['evaluate']

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.4828441558441558
Score: 0.4828441558441558
Sensibilidade: 0.4737662337662338
Especificidade: 0.4919220779220779 

person_17    0.051492
person_12    0.049491
person_11    0.049309
person_21    0.047853
person_8     0.047489
person_16    0.046397
person_4     0.046397
person_18    0.046215
person_6     0.045852
person_10    0.045488
person_15    0.045488
person_7     0.045306
person_3     0.044942
person_19    0.044942
person_5     0.044578
person_1     0.044214
person_14    0.043850
person_0     0.043304
person_20    0.042940
person_2     0.042394
person_9     0.041303
person_13    0.040757
dtype: float64


### 2.1.3 Variáveis categórias sendo dummy

In [6]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()
Y = df_parkinson['drug']

#Criando dummies
X = pd.concat([X,pd.get_dummies(df_parkinson['measure'],prefix='measure'),pd.get_dummies(df_parkinson['evaluate'],prefix='evaluate')],axis=1)

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.4855974025974026
Score: 0.4855974025974026
Sensibilidade: 0.47979220779220777
Especificidade: 0.49140259740259734 

person_20    0.048763
person_10    0.048581
person_4     0.048035
person_3     0.047853
person_21    0.047489
person_13    0.047307
person_14    0.047125
person_5     0.046761
person_18    0.046579
person_2     0.045670
person_0     0.045488
person_8     0.045306
person_1     0.045124
person_7     0.045124
person_15    0.044396
person_11    0.043850
person_17    0.043850
person_6     0.043122
person_16    0.042940
person_19    0.042576
person_12    0.042394
person_9     0.041667
dtype: float64


### 2.1.3 Váriveis categóricas sendo dummy, porém unidas

In [7]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()

#Criando dummies
X = pd.concat([X,pd.get_dummies(df_parkinson['measure'] + '_' + df_parkinson['evaluate'].map(lambda x: str(x)))],axis=1)

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.5021168831168832
Score: 0.5021168831168832
Sensibilidade: 0.4995844155844155
Especificidade: 0.5046493506493506 

person_3     0.048745
person_9     0.048563
person_19    0.047472
person_11    0.047108
person_10    0.047108
person_2     0.046199
person_0     0.046199
person_8     0.045835
person_12    0.045653
person_21    0.045653
person_4     0.045289
person_5     0.045289
person_1     0.045107
person_20    0.044744
person_16    0.044562
person_15    0.044562
person_14    0.044562
person_13    0.044016
person_17    0.043652
person_18    0.043470
person_7     0.043107
person_6     0.043107
dtype: float64


A utilização de vários tipos de codificação para a variável fase e medida foram tentados por nossas análises mostrarem que existem diferenças dependendo da onde você olha.

Contudo, independente da forma de codificação utilizada não conseguimos bons resultados. Esse fato ocorreu mesmo havendo uma variação dos algoritmos supervisionados.

Acreditamos que essas codificações não derão certo por em cada fase/medida possuir um comportamente diferente. Vamos avaliar essa hipótese na próxima seção.

### 2.1.4 Classificação considerando apenas uma fase e medida

In [8]:
#Filtro selecionado a medida P na avaliação 1 com PC4 e PC5
df_parkinson = df_parkinson.loc[(df_parkinson['measure'] == 'P') & (df_parkinson['evaluate'] == 1),['PC4','PC5','drug']]

In [9]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.loc[:,['PC4','PC5']]
Y = df_parkinson['drug']

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.7513039682539683
Score: 0.733090909090909
Sensibilidade: 0.7125373015873016
Especificidade: 0.790070634920635 

person_8     0.149864
person_0     0.117847
person_15    0.081744
person_11    0.080381
person_7     0.079019
person_2     0.075613
person_17    0.072888
person_18    0.055177
person_6     0.053134
person_19    0.049046
person_3     0.029292
person_9     0.029292
person_21    0.027929
person_1     0.026567
person_16    0.020436
person_14    0.018392
person_12    0.016349
person_20    0.009537
person_13    0.002725
person_5     0.002044
person_4     0.002044
person_10    0.000681
dtype: float64


Podemos ver que agora obtivemos bons resultados. Contudo isso foi a custa de uma grande diminuição da quantidade de amostras no dataset.

Salientamos também que esse valor só foi possível de alcançar utilizando o PC4 e PC5. Qualquer outro componente utilizado acaba 'atrapalhando' na classificação.

### 2.1.5 Uso de novas features

Dado que não foi possível obter bons resultados com as features anteriores, iremos utilizar novos atributos para averiguar se conseguimos melhores valores.

As novas features calculadas foram baseadas em https://github.com/Luke3D/CIS_PD-NDM que fez um trabalho similar com dados de Parkinson.

In [10]:
#Setando diretorio Data como o atual
os.chdir('../Data')

#Leitura
df_parkinson = pd.read_csv('parkinson_v2.csv',index_col = 'name')

#Apresentação de algumas linhas
df_parkinson.head()

,pse,pspf1,pspf2,pspf3,psp1,psp2,psp3,wpsf,RMS,range,...,var,skew,kurt,jerk_mean,jerk_std,jerk_skew,jerk_kur,evaluate,measure,drug
name,,,,,,,,,,,,,,,,,,,,,
person_0,3.021855,1,8,17,17243.324518,1850.713750,1693.279040,17243.324518,382.050996,-6848.679085,...,363.377887,1.412557,30.949340,0.050528,535.257543,-0.163219,30.336621,2,B,placebo
person_0,3.533303,1,3,6,39115.767987,13796.968242,5978.057257,39115.767987,390.853005,-3097.095783,...,908.940592,0.098040,4.969888,-0.922482,1153.757327,-0.175426,6.861516,2,P,placebo
person_0,0.859185,1,4,13,129866.558198,3193.568772,1810.657041,129866.558198,397.813776,-6028.623410,...,630.078604,0.073222,2.749642,-0.111951,814.505179,0.168532,3.834549,2,A,placebo
person_0,3.424263,1,4,7,102144.341817,60726.093359,42408.550846,102144.341817,470.023084,32858.801543,...,4939.505835,1.007991,0.771216,1.427418,6544.802604,0.251431,0.646722,2,S1,placebo
person_0,2.009236,1,3,5,150065.780097,50565.746519,23406.888141,150065.780097,477.519118,10429.318103,...,4692.390496,2.259779,6.046340,2.919689,6994.739267,-0.003559,3.055311,2,S2,placebo


Explicação das novas features:

- jerk_(mean|std|skew|kur): Média, desvio padrão, assimetria e curtose para a série temporal com lag1.
- RMS: Root mean square da série temporal.
- range: Intervalo da série temporal.
- mean: Médida da série temporal.
- var: Variância da série temporal.
- skew: Assimetria da série temporal.
- kurt: Curtose da série temporal.

Vamos fazer algumas mudanças nos dados como criar novas colunas e aplicar a boxcox para normalizar.

In [11]:
#Mapenado droga como variavel numerica
df_parkinson['drug'] = LabelEncoder().fit_transform(df_parkinson['drug'])

#Criacao de novas colunas
df_parkinson['wpsf2'] = df_parkinson['pspf2'] * df_parkinson['psp2']
df_parkinson['wpsf3'] = df_parkinson['pspf3'] * df_parkinson['psp3']
df_parkinson.rename(columns={'wpsf':'wpsf1'},inplace=True)

#Arrumando posicao das colunas
cols = list(df_parkinson.columns[0:8]) + ['wpsf2','wpsf3','jerk_mean', 'jerk_std','jerk_skew', 'jerk_kur','RMS','range', 'mean', 'var', 'skew', 'kurt','evaluate','measure','drug']
df_parkinson = df_parkinson.loc[:,cols]

#Aplicando log
positive_cols = (df_parkinson <= 0).any()
positive_cols = positive_cols[~positive_cols].index
positive_cols = positive_cols.drop('evaluate')
df_parkinson.loc[:,positive_cols] = np.log(df_parkinson.loc[:,positive_cols])

#Reescalando variaveis
mm = MinMaxScaler().fit(df_parkinson.iloc[:,:-3])
df_parkinson.iloc[:,:-3] = mm.transform(df_parkinson.iloc[:,:-3])

Como possuímos 18 atríbutos, utilizararemos o <b>forward selection</b> para selecionar as melhores features.

In [12]:
#Arvore de decisao sera usada
dt = DecisionTreeClassifier()

#Criando foward selection
sfs1 = sfs(dt,k_features=3,forward=True,floating=False,scoring='roc_auc',cv=5)

#Divisao do dataset
X = df_parkinson.iloc[:,:-3]
Y = df_parkinson['drug']

#Selecao
sfs1 = sfs1.fit(X,Y)
print(sfs1.k_feature_names_)

#Classificacao
evaluate_classifier(X.loc[:,sfs1.k_feature_names_],Y)

('wpsf2', 'jerk_skew', 'jerk_kur')
AUC: 0.5375844155844156
Score: 0.5375844155844155
Sensibilidade: 0.5271168831168832
Especificidade: 0.548051948051948 

person_7     0.049273
person_9     0.049091
person_16    0.048727
person_18    0.048000
person_6     0.047091
person_20    0.047091
person_13    0.046545
person_17    0.046364
person_19    0.046182
person_12    0.046000
person_5     0.045636
person_3     0.045636
person_2     0.045273
person_8     0.044727
person_0     0.044545
person_11    0.044182
person_1     0.044000
person_14    0.044000
person_10    0.043636
person_4     0.043091
person_15    0.042727
person_21    0.038182
dtype: float64


Em vários testes realizados manualmente, vimos que não foi possível obter bons resultados. 

Assim, utilizamos a mesma ideia anterior de realizar uma filtragem por fase/medida.

Com a ajuda do forward selection concluímos que para cada fase/medida deveríamos utilizar os seguintes atributos:

| Medida       | Fase     | Features     |
| :------------- | :----------: | -----------: |
| P | 1 | psp3, pspf3, jerk_mean|
| P | 2 | psp1, pspf1, jerk_mean|
| B | 1 | pspf3, wpsf3, jerk_std, range|
| B | 2 | pspf1, pspf3, var, jerk_std|
| A | 1 | Nenhum|
| A | 2 | wpsf2, kurt, jerk_kur|
| S1 | 1 | pse, kurt, jerk_mean|
| S1 | 2 | Nenhum|
| S2 | 1 | Nenhum|
| S2 | 2 | psp1, psp3|
| F1 | 1 | psp2, jerk_kur|
| F1 | 2 | pspf2, kurt, jerk_kur|
| F2 | 1 | pspf1, kurt|
| F2 | 2 | pspf1, pspf2|


Sabemos por trabalhos similares que a classificação utilizando dados de acelerômetro para o problema de predizer pessoas com/sem Parkinson são bem difíceis. Por exemplo, conforme podemos ver aqui https://blog.insightdatascience.com/head-over-heels-detecting-parkinsons-disease-from-accelerometer-data-b36aa46e320b, houve uma dificuldade muito grande em obter bons atributos sendo que muitos deles tinham uma enorme sobreposição de classes.

Sendo assim, nós tivemos as mesmas dificuldades mesmo que o nosso problema seja um pouco diferente. Em várias de nossas tentativas não foi possível obter resultados satisfatórios.

A única 'solução' encontrada foi utilizar filtros nas fases/medidas conforme a tabela acima. Contudo, essa proposta não é viável, já que, a utilização da filtagrem acarreta em pouquíssimos dados, não sendo possível validar os modelos.

É interessante notar, portanto, que segundo nossas análises temos que para cada fase/medida as features se comportam de uma maneira diferente, ou seja, dependendo da situação a distribuição delas no espaço muda. Isso pode ter acontecido pelo fato de que em cada fase nós temos níveis de estresse diferente. A mudança nas medidas pode ter envolvimento com o fator psicológico, já que, na segunda avaliação as pessoas estavam mais preparadas para o discurso. 

Nota-se também o porque da utilização de várias codificações não terem dado certo. Como cada fase/medida possui seu própio conjunto ótimo de features, mesmo que tentassemos realizar uma separação espacial nos dados a mistura de atributos ocasionava confusões no algoritmo.

Portanto, concluímos que com os dados disponíveis não é possível realizar uma classificação satisfatória. Futuramente, o uso de novas features pode resolver o problema. Caso o contrário, a sugestão seria obter mais dados para que as avaliações fossem mais precisas. 

Mesmo não sendo ideal para validação, abaixo temos uma demo de como são os resultados da classificação utilizando a filtragem.

In [13]:
# FILTRO - TESTE
df_test = df_parkinson.loc[(df_parkinson['measure'] == 'P') & (df_parkinson['evaluate'] == 2),['psp1','pspf1','jerk_mean','drug']]

#Separando dataset em variaveis  dependentes e independentes
X = df_test.select_dtypes('float')
Y = df_test['drug']

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.7944746031746032
Score: 0.78
Sensibilidade: 0.8007222222222222
Especificidade: 0.7882269841269842 

person_14    0.180992
person_10    0.102479
person_16    0.098347
person_0     0.093388
person_7     0.086777
person_11    0.065289
person_21    0.060331
person_2     0.057851
person_8     0.042975
person_18    0.033058
person_19    0.030579
person_9     0.023967
person_3     0.021488
person_20    0.016529
person_17    0.016529
person_5     0.015702
person_13    0.013223
person_4     0.011570
person_15    0.009917
person_12    0.009091
person_1     0.004959
person_6     0.004959
dtype: float64


## 3. Conclusão

- Não foi possível obter bons resultados.
- Causas: Grande aleatoriedade nos dados pelo fato dos diferentes níveis de estresse que as pessoas passavam.
- Possíveis soluções futuras: Descoberta de novos atributos e mais dados.

## 4. Observações

Como não foi possível obter resultados satisfatórios com a classificação, certamente o uso de um algoritmo não supervisionado para descrever os dados iria agir de maneira menos precisa, já que, o poder dos algoritmos superviosionados tende a ser maior.

Assim, não iremos nos ater a criar ou realizar análises de clustering com esses dados.